In [1]:
import pandas as pd
import numpy as np

from ta import momentum, trend, volume
from binance import Client
from key import PUB_KEY, SEC_KEY

# Get Historical Data

In [2]:
# Connecting to Binance
client = Client(PUB_KEY, SEC_KEY)
# Get historical data
df = pd.DataFrame(client.get_historical_klines('BTCUSDT',
                             Client.KLINE_INTERVAL_1MINUTE,
                                      str(3000) + ' minutes ago UTC'))

# First 6 column represent Time and OHLCV values
df = df.iloc[:, :6]
df.columns = ['Time', 'Open', 'High', 'Low', 'Close', 'Volume']

# Convert first column into datetime
df['Time'] = pd.to_datetime(df['Time'], unit='ms')
for column in df.columns:
    if column != 'Time':
        df[column] = df[column].astype(float)

# Set Time as index
df.set_index('Time', inplace=True)

# Generate Technical Indicators

In [3]:
df

,Open,High,Low,Close,Volume
Time,,,,,
2024-07-15 15:59:00,63017.39,63030.00,63011.36,63026.00,10.74969
2024-07-15 16:00:00,63026.01,63026.01,62984.47,63025.98,27.53709
2024-07-15 16:01:00,63025.99,63061.64,63018.00,63022.00,30.12760
2024-07-15 16:02:00,63022.01,63022.01,62956.00,62956.01,20.16122
2024-07-15 16:03:00,62956.00,63015.33,62956.00,62981.60,29.28594
...,...,...,...,...,...
2024-07-17 17:54:00,64304.01,64318.00,64296.27,64312.00,16.91624
2024-07-17 17:55:00,64312.00,64341.28,64312.00,64334.01,14.56475
2024-07-17 17:56:00,64334.01,64353.33,64330.00,64344.00,24.52004


In [4]:
df['SMA50'] = df['Close'].rolling(50).mean()
df['SMA200'] = df['Close'].rolling(200).mean()
df['RSI'] = momentum.rsi(df['Close'])
df['MACD'] = trend.macd(df['Close'])
df['MACD_sig'] = trend.macd_signal(df['Close'])
df['MACD_diff'] = trend.macd_diff(df['Close'])
df['Momentum'] = (df['Close'] - df['Close'].shift(30))/df['Close'].shift(30)
df['ADX'] = trend.adx(df['High'], df['Low'], df['Close'])
df['-DI'] = trend.adx_neg(df['High'], df['Low'], df['Close'])
df['+DI'] = trend.adx_pos(df['High'], df['Low'], df['Close'])


# Generate Trading Signals

For each TAs, a buy, hold and sell signal will be generated. They will bear the weight of 1, 0 and -1 respectively

In [5]:
df.dropna(inplace=True)
df

,Open,High,Low,Close,Volume,SMA50,SMA200,RSI,MACD,MACD_sig,MACD_diff,Momentum,ADX,-DI,+DI
Time,,,,,,,,,,,,,,,
2024-07-15 19:18:00,63555.85,63566.17,63533.83,63563.49,28.73448,63499.2438,63360.32475,55.265820,27.499883,19.310349,8.189535,0.001473,17.331435,21.647975,32.644341
2024-07-15 19:19:00,63563.48,63563.49,63511.13,63553.99,16.52164,63501.4810,63362.96470,54.089510,26.062713,20.660821,5.401891,0.001283,17.059392,23.148304,30.387920
2024-07-15 19:20:00,63553.98,63620.24,63546.73,63600.01,42.83555,63504.5186,63365.83485,58.677850,28.310824,22.190822,6.120002,0.001601,17.615740,20.958050,34.817238
2024-07-15 19:21:00,63600.01,63777.00,63600.00,63719.08,158.55130,63509.7924,63369.32025,67.678527,39.247996,25.602257,13.645740,0.002749,19.637331,16.829037,45.406252
2024-07-15 19:22:00,63719.08,63780.00,63657.56,63728.00,111.18273,63513.0614,63373.18020,68.236696,48.081302,30.098066,17.983236,0.003591,21.536717,14.675194,39.908575
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-17 17:54:00,64304.01,64318.00,64296.27,64312.00,16.91624,64211.9476,64595.77435,52.791641,19.755919,31.525868,-11.769949,0.002679,18.256143,22.045753,27.015750
2024-07-17 17:55:00,64312.00,64341.28,64312.00,64334.01,14.56475,64213.9878,64591.40440,55.312477,20.061870,29.233068,-9.171198,0.002962,18.125148,21.067082,29.346032
2024-07-17 17:56:00,64334.01,64353.33,64330.00,64344.00,24.52004,64218.4680,64587.31935,56.449193,20.869874,27.560430,-6.690555,0.003493,18.227745,20.294031,30.164479


In [6]:
#SMA Golden Cross & Death Cross
sma_buy_cond = (df['SMA50'] > df['SMA200']) & (df['SMA50'].shift(1) < df['SMA200'])
sma_sell_cond = (df['SMA50'] < df['SMA200']) & (df['SMA50'].shift(1) > df['SMA200'])
df['Signal_sma'] = np.where(sma_buy_cond, 1, np.where(sma_sell_cond, -1, 0))

In [7]:
# MACD Below 0 Cross
macd_buy_cond = (df['MACD_diff'] > 0) & (df['MACD_diff'].shift(1) < 0) & (df['MACD'] < 0) &(df['MACD_sig'] < 0)
macd_sell_cond = (df['MACD_diff'] < 0) & (df['MACD_diff'].shift(1) > 0) & (df['MACD'] > 0) &(df['MACD_sig'] > 0)
df['Signal_macd'] = np.where(macd_buy_cond, 1, np.where(macd_sell_cond, -1, 0))

In [8]:
# RSI Overbought & Oversold
rsi_buy_cond = (df['RSI'] > 30) & (df['RSI'].shift(1) < 30)
rsi_sell_cond = (df['RSI'] < 70) & (df['RSI'].shift(1) > 70)
df['Signal_rsi'] = np.where(rsi_buy_cond, 1, np.where(rsi_sell_cond, -1, 0))

In [9]:
df

,Open,High,Low,Close,Volume,SMA50,SMA200,RSI,MACD,MACD_sig,MACD_diff,Momentum,ADX,-DI,+DI,Signal_sma,Signal_macd,Signal_rsi
Time,,,,,,,,,,,,,,,,,,
2024-07-15 19:18:00,63555.85,63566.17,63533.83,63563.49,28.73448,63499.2438,63360.32475,55.265820,27.499883,19.310349,8.189535,0.001473,17.331435,21.647975,32.644341,0,0,0
2024-07-15 19:19:00,63563.48,63563.49,63511.13,63553.99,16.52164,63501.4810,63362.96470,54.089510,26.062713,20.660821,5.401891,0.001283,17.059392,23.148304,30.387920,0,0,0
2024-07-15 19:20:00,63553.98,63620.24,63546.73,63600.01,42.83555,63504.5186,63365.83485,58.677850,28.310824,22.190822,6.120002,0.001601,17.615740,20.958050,34.817238,0,0,0
2024-07-15 19:21:00,63600.01,63777.00,63600.00,63719.08,158.55130,63509.7924,63369.32025,67.678527,39.247996,25.602257,13.645740,0.002749,19.637331,16.829037,45.406252,0,0,0
2024-07-15 19:22:00,63719.08,63780.00,63657.56,63728.00,111.18273,63513.0614,63373.18020,68.236696,48.081302,30.098066,17.983236,0.003591,21.536717,14.675194,39.908575,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-07-17 17:54:00,64304.01,64318.00,64296.27,64312.00,16.91624,64211.9476,64595.77435,52.791641,19.755919,31.525868,-11.769949,0.002679,18.256143,22.045753,27.015750,0,0,0
2024-07-17 17:55:00,64312.00,64341.28,64312.00,64334.01,14.56475,64213.9878,64591.40440,55.312477,20.061870,29.233068,-9.171198,0.002962,18.125148,21.067082,29.346032,0,0,0
2024-07-17 17:56:00,64334.01,64353.33,64330.00,64344.00,24.52004,64218.4680,64587.31935,56.449193,20.869874,27.560430,-6.690555,0.003493,18.227745,20.294031,30.164479,0,0,0
